**129L Section 7B** <br>
2/20/25

In [1]:
import numpy as np

# Task 1: Sobol Sequence

# Task 2: Monte Carlo Integration vs. Deterministic Quadrature

**Part a)**

**Part b)**

**Part c)**

**Part d)**

**Part e)**

**Part f)**

# Task 3: Rejection Sampling

**Part a)**

**Part b)**